## CREATE FS-SURFACES FROM NMT-SEGMENTED ANATOMICALS

### Initiate files and paths

In [ ]:
# Subject name
SUBJ=Danny

# paths
src_path=~/Desktop/FS_ALT/${SUBJ}
src_vol=${src_path}/${SUBJ}_N4
src_brain=${src_path}/${SUBJ}_brain
src_brainmask=${src_path}/${SUBJ}_brainmask
src_WM=${src_path}/${SUBJ}_WM
out_path=~/Desktop/FS_ALT/${SUBJ}/fs
mkdir -p ${out_path}

# facilitate doing hemispheres in a for loop
HEMI=(lh rh)

### Change headers to pretend we have 1 mm isotropic voxels

In [ ]:
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen $src_vol.nii.gz
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen $src_brain.nii.gz
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen $src_brainmask.nii.gz
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen $src_WM.nii.gz

### Convert volumes to mgz

In [ ]:
mri_convert -c $src_vol.nii.gz $src_vol.mgz
mri_convert -c $src_brain.nii.gz $src_brain.mgz
mri_convert -c $src_brainmask.nii.gz $src_brainmask.mgz
mri_convert -c $src_WM.nii.gz $src_WM.mgz

### Get corpus callosum and pons coordinates & fill WM

In [ ]:
# Inspect volume to get voxel coordinates
freeview -v $src_brain.mgz &

In [ ]:
# Fill in WM and check result
CC=(127 114 129)
PONS=(124 148 129)

In [ ]:
# Fill WM
mri_fill -CV ${CC[0]} ${CC[1]} ${CC[2]} \
    -PV ${PONS[0]} ${PONS[1]} ${PONS[2]} \
    $src_WM.mgz $out_path/WM_filled.mgz

# Check the result
freeview $out_path/WM_filled.mgz &

In [ ]:
# copy the white matter volume so we can edit it later
cp $src_WM.mgz $out_path/${SUBJ}_WM_fix.mgz

In [ ]:
mri_mask -T 5 $src_brain.mgz $src_brainmask.mgz ${src_path}/${SUBJ}_brain.finalsurfs.mgz

### Tesselate volumes (first run)

In [ ]:
# left
mri_pretess $out_path/WM_filled.mgz 255 $src_brain.mgz $out_path/WM_filled-pretess255.mgz
mri_tessellate $out_path/WM_filled-pretess255.mgz 255 $out_path/lh.orig.nofix
# right
mri_pretess $out_path/WM_filled.mgz 127 $src_brain.mgz $out_path/WM_filled-pretess127.mgz
mri_tessellate $out_path/WM_filled-pretess127.mgz 127 $out_path/rh.orig.nofix

for xh in ${HEMI[@]}; do
    # -- Tesselate hemispheres --
    # NB! you may need to redo from here after fixing WM
    mris_extract_main_component $out_path/${xh}.orig.nofix $out_path/${xh}.orig.nofix
    mris_smooth -nw $out_path/${xh}.orig.nofix $out_path/${xh}.smoothwm.nofix
    mris_inflate $out_path/${xh}.smoothwm.nofix $out_path/${xh}.inflated.nofix
    mris_sphere -q $out_path/${xh}.inflated.nofix $out_path/${xh}.qsphere.nofix
done

In [ ]:
# Inspect the results
freeview -v $src_brain.mgz \
    -f $out_path/lh.smoothwm.nofix $out_path/lh.inflated.nofix \
    $out_path/rh.smoothwm.nofix $out_path/rh.inflated.nofix &

In [ ]:
# Fix topology
for xh in ${HEMI[@]}; do
    cp $out_path/${xh}.orig.nofix $out_path/${xh}.orig
    cp $out_path/${xh}.inflated.nofix $out_path/${xh}.inflated
    mris_euler_number $out_path/${xh}.orig
    mris_remove_intersection $out_path/${xh}.orig $out_path/${xh}.orig
    mris_smooth -nw $out_path/${xh}.orig $out_path/${xh}.smoothwm
    mris_inflate $out_path/${xh}.smoothwm $out_path/${xh}.inflated
done

### Manual adjustments and re-tesselations

In [ ]:
# Check results again and edit the WM volume to fix errors
# keep doing this until you're happy with the result
freeview -v $src_brain.mgz -v $out_path/${SUBJ}_WM_fix.mgz \
    -f $out_path/lh.smoothwm $out_path/lh.inflated \
    $out_path/rh.smoothwm $out_path/rh.inflated &

In [ ]:
# redo the tesselation with the fixed WM volume
mri_fill -CV ${CC[0]} ${CC[1]} ${CC[2]} \
    -PV ${PONS[0]} ${PONS[1]} ${PONS[2]} \
    $out_path/${SUBJ}_WM_fix.mgz $out_path/WM_filled.mgz
    
mri_pretess $out_path/WM_filled.mgz 255 $src_brain.mgz $out_path/WM_filled-pretess255.mgz
mri_tessellate $out_path/WM_filled-pretess255.mgz 255 $out_path/lh.orig.nofix
mri_pretess $out_path/WM_filled.mgz 127 $src_brain.mgz $out_path/WM_filled-pretess127.mgz
mri_tessellate $out_path/WM_filled-pretess127.mgz 127 $out_path/rh.orig.nofix

for xh in ${HEMI[@]}; do
    mris_extract_main_component $out_path/${xh}.orig.nofix $out_path/${xh}.orig.nofix
    mris_smooth -nw $out_path/${xh}.orig.nofix $out_path/${xh}.smoothwm.nofix
    mris_inflate $out_path/${xh}.smoothwm.nofix $out_path/${xh}.inflated.nofix
    mris_sphere -q $out_path/${xh}.inflated.nofix $out_path/${xh}.qsphere.nofix
    
    cp $out_path/${xh}.orig.nofix $out_path/${xh}.orig
    cp $out_path/${xh}.inflated.nofix $out_path/${xh}.inflated
    
    mris_euler_number $out_path/${xh}.orig
    mris_remove_intersection $out_path/${xh}.orig $out_path/${xh}.orig
    mris_smooth -nw $out_path/${xh}.orig $out_path/${xh}.smoothwm
    mris_inflate $out_path/${xh}.smoothwm $out_path/${xh}.inflated
    mris_curvature -thresh .999 -n -a 5 -w -distances 10 10 $out_path/${xh}.inflated
done
echo 'TESSELATION DONE!'

### Next steps

In [ ]:
# create a freesurfer SUBJECT with the correct folder structure
echo 'Creating a subject directory for '${SUBJ}'in:'
echo ${SUBJECTS_DIR}/${SUBJ}
mksubjdirs ${SUBJECTS_DIR}/${SUBJ}

In [ ]:
# copy the necessary files for cutting to the subject directory
cp $out_path/lh.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/
cp $out_path/lh.smoothwm ${SUBJECTS_DIR}/${SUBJ}/surf/
cp $out_path/lh.orig ${SUBJECTS_DIR}/${SUBJ}/surf/

cp $src_path/${SUBJ}_N4.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/T1.mgz
cp $out_path/WM_filled.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/filled.mgz
cp $out_path/${SUBJ}_WM_fix.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/wm.mgz
cp ${src_path}/${SUBJ}_brain.finalsurfs.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/brain.finalsurfs.mgz

In [ ]:
xh=${HEMI[0]} # 1 hemisphere for testing
echo ${xh}
mris_make_surfaces -noaseg -noaparc -T1 brain.finalsurfs -orig_wm orig ${SUBJ} ${xh}

In [ ]:
freeview ${SUBJ} -v T1 -f lh.inflated

In [ ]:
tksurfer ${SUBJ} lh inflated 

In [ ]:
cd ${SUBJECTS_DIR}/${SUBJ}/surf/
mris_flatten -w 0 -distances 20 7 lh.full.patch.3d  lh.full.patch.flat
